In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 # These are the Libraries that is required to run this notebook

In [ ]:
from crewai import Agent, Task, Crew # These are the Things that is require to build AI Agents using Crew, Creating Agents using Agent Instance, Task Instance and Assembling all things using Crew

In [ ]:
import os
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

os.environ["OPENAI_MODEL_NAME"]='gpt-3.5-turbo'
os.environ["OPENAI_API_KEY"] = openai_api_key # Here Configuring Everything like Model and other API keys that we gonna use in Tools

In [ ]:
support_agent = Agent( # Defining Agent with Specific Goals, Roles and Backstory which will play a role of Agent who have persona to act like and behave to complete given Task.
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False, # This is by default True , but here it is false because we don't want this agent to delegate the task taht is given to him
	verbose=True # For detailing out the Execution and we can see log of Everything
)



*   By not setting allow_delegation=False, allow_delegation takes its default value of being True.
* This means the agent can delegate its work to another agent which is better suited to do a particular task.




In [ ]:
support_quality_assurance_agent = Agent( # This is the Second Agent for Quality Assurance to verify , everything is going right.
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True # For detailing out the Execution and we can see log of Everything and this agent can delgate task to other AGent
)

In [ ]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool # importing other tools to carry out Tasks, Tools are the real things which have real Impact on use case.

In [ ]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

In [ ]:
# Here we are Defining Task that will carry out by Agents and in this Task Description and Expected OUtput criteria is set.
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool], # here Clearly mentioning that which tools this task will use and Task's tool override Tools posses by Agents to perform that task.
    agent=support_agent, # Here Defining which agent will perform this Task.
)

In [ ]:
quality_assurance_review = Task( # This is the Task that Quality Assurance Agent will perform
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)



In [ ]:
# Here created Instance of Crew which is used to assemble everything in a unt Agents, Tools and tasks and Adding Memmory.
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

In [ ]:
# giving Input to Agent to Perform its task
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

In [ ]:
# using Markdown to display Answer in more Beautiful and better format.
from IPython.display import Markdown
Markdown(result)